## Notebook 3: Urban Heat Islands in St. Louis

**Why is downtown 10°F hotter than Forest Park?**

Cities create their own microclimates. Concrete, asphalt, and buildings absorb heat during the day and release it slowly at night. Meanwhile, parks and suburbs with trees stay cooler through shade and evapotranspiration.

This effect—the **urban heat island**—is measurable from space. In this notebook, we'll:

- Map land surface temperature across St. Louis using MODIS satellite data
- Compare downtown, Forest Park, and the suburbs
- See how vegetation correlates with cooler temperatures


## Setup

Same initialization as previous notebooks.

In [10]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [11]:
import ee
from google.cloud import storage

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

Ready!


## Define area of interest: St. Louis Metro

We'll use St. Louis City and St. Louis County from the TIGER census boundaries to capture the urban core and surrounding suburbs.

In [12]:
# Get St. Louis City and County from TIGER boundaries
counties = ee.FeatureCollection("TIGER/2018/Counties")

# St. Louis City (independent city, FIPS 29510) and St. Louis County (FIPS 29189)
stl_city = counties.filter(ee.Filter.eq("STATEFP", "29")).filter(ee.Filter.eq("NAME", "St. Louis city"))
stl_county = counties.filter(ee.Filter.eq("STATEFP", "29")).filter(ee.Filter.eq("NAME", "St. Louis"))

# Merge into single region
stl_metro = stl_city.merge(stl_county)
stl_geometry = stl_metro.geometry()

# Define key landmarks for reference
landmarks = {
    "Downtown (Gateway Arch)": [-90.1848, 38.6247],
    "Forest Park": [-90.2845, 38.6370],
    "Lambert Airport": [-90.3599, 38.7487],
    "Tower Grove Park": [-90.2543, 38.6046],
}

print(f"Area of interest defined: St. Louis City + County")

Area of interest defined: St. Louis City + County


## Load MODIS Land Surface Temperature

MODIS (Moderate Resolution Imaging Spectroradiometer) provides land surface temperature data twice daily at 1km resolution. The MOD11A2 product gives us 8-day composites, which helps reduce cloud contamination.

We'll filter to summer 2024 (June-August) when heat island effects are most pronounced.

In [13]:
# MODIS Land Surface Temperature (8-day composite)
modis_lst = ee.ImageCollection("MODIS/061/MOD11A2")

# Filter to summer 2024
summer_start = "2024-06-01"
summer_end = "2024-08-31"

lst_summer = (
    modis_lst
    .filterDate(summer_start, summer_end)
    .filterBounds(stl_geometry)
    .select("LST_Day_1km")  # Daytime land surface temperature
)

# Create median composite
lst_median = lst_summer.median().clip(stl_geometry)

print(f"Loaded {lst_summer.size().getInfo()} MODIS scenes from summer 2024")

Loaded 12 MODIS scenes from summer 2024


## Convert from Kelvin to Fahrenheit

MODIS stores temperature in Kelvin, scaled by 0.02. We'll convert to Fahrenheit for a US audience:

- First multiply by 0.02 to get Kelvin
- Convert to Celsius: (K - 273.15)
- Convert to Fahrenheit: (C * 9/5) + 32

In [14]:
def kelvin_to_fahrenheit(img):
    """Convert MODIS LST from scaled Kelvin to Fahrenheit."""
    # Scale factor is 0.02 for MODIS LST
    kelvin = img.multiply(0.02)
    celsius = kelvin.subtract(273.15)
    fahrenheit = celsius.multiply(9/5).add(32)
    return fahrenheit

lst_fahrenheit = kelvin_to_fahrenheit(lst_median)

print("Temperature converted to Fahrenheit")

Temperature converted to Fahrenheit


## Visualize the Urban Heat Island

We'll use a heat palette from cool blues to hot reds. Look for:
- **Hot zones** (red): Downtown, industrial areas, Lambert Airport runways
- **Cool zones** (blue): Forest Park, Tower Grove Park, residential areas with trees

In [15]:
# Create map centered on St. Louis
m = geemap.Map(center=[38.65, -90.3], zoom=10)

# Heat palette: blue (cool) to red (hot)
heat_palette = [
    '#313695',  # dark blue (coolest)
    '#4575b4',  # blue
    '#74add1',  # light blue
    '#abd9e9',  # pale blue
    '#fee090',  # pale yellow
    '#fdae61',  # orange
    '#f46d43',  # red-orange
    '#d73027',  # red
    '#a50026',  # dark red (hottest)
]

# Visualization parameters (typical summer surface temps in F)
temp_vis = {
    "min": 85,
    "max": 115,
    "palette": heat_palette
}

# Add temperature layer
m.addLayer(lst_fahrenheit, temp_vis, "Land Surface Temperature (°F)")

# Add boundary
m.addLayer(stl_geometry, {"color": "black"}, "St. Louis Metro Boundary")

# Add landmark markers
for name, coords in landmarks.items():
    point = ee.Geometry.Point(coords)
    m.addLayer(point, {"color": "white"}, name)

# Add colorbar legend
m.add_colorbar(temp_vis, label="Surface Temperature (°F)", layer_name="Land Surface Temperature (°F)")

m

## Vegetation and Temperature: Side-by-Side Comparison

There's a strong inverse relationship between vegetation (NDVI) and surface temperature. Areas with more trees and grass are cooler.

Let's load NDVI from the same summer period and compare them side by side.

In [16]:
# Sentinel-2 for NDVI
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

def mask_s2_clouds(img):
    """Mask clouds using Scene Classification Layer."""
    scl = img.select("SCL")
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return img.updateMask(mask)

def add_ndvi(img):
    """Calculate NDVI."""
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

# Get summer NDVI composite
ndvi_summer = (
    s2.filterBounds(stl_geometry)
      .filterDate(summer_start, summer_end)
      .map(mask_s2_clouds)
      .map(add_ndvi)
      .select("NDVI")
      .median()
      .clip(stl_geometry)
)

print("NDVI composite created")

NDVI composite created


In [17]:
# NDVI visualization
ndvi_palette = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
ndvi_vis = {"min": 0.0, "max": 0.8, "palette": ndvi_palette}

# Create split map: Temperature on left, NDVI on right
left_layer = geemap.ee_tile_layer(lst_fahrenheit, temp_vis, "Temperature (°F)")
right_layer = geemap.ee_tile_layer(ndvi_summer, ndvi_vis, "NDVI (Vegetation)")

m2 = geemap.Map(center=[38.63, -90.28], zoom=11)
m2.split_map(left_layer, right_layer)
m2

## What do you notice?

Drag the slider on the split map and look for patterns:

- **Forest Park** appears green (high NDVI) on the right and blue (cool) on the left
- **Downtown** and industrial areas are brown/red (low NDVI) and orange/red (hot)
- **Lambert Airport** runways show up clearly as hot spots with no vegetation
- **Residential areas** vary—older neighborhoods with mature trees are cooler than newer developments

This inverse relationship between vegetation and temperature is why urban forestry and green infrastructure are key climate adaptation strategies.

## Sample temperatures at specific locations

Let's extract actual temperature values at our landmarks to quantify the urban heat island effect.

In [18]:
print("Summer 2024 Average Daytime Surface Temperatures:")
print("=" * 50)

for name, coords in landmarks.items():
    point = ee.Geometry.Point(coords)
    # Sample the temperature at this point
    temp_value = lst_fahrenheit.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=1000  # MODIS resolution
    ).get("LST_Day_1km")
    
    temp_f = temp_value.getInfo()
    if temp_f:
        print(f"{name:30} {temp_f:.1f}°F")

print("=" * 50)
print("\nNotice the difference between parks and urban areas!")

Summer 2024 Average Daytime Surface Temperatures:
Downtown (Gateway Arch)        97.2°F
Forest Park                    91.7°F
Lambert Airport                95.6°F
Tower Grove Park               96.7°F

Notice the difference between parks and urban areas!


Forest Park                    91.7°F


Lambert Airport                95.6°F


Tower Grove Park               96.7°F

Notice the difference between parks and urban areas!


## Try it yourself

Here are some ideas to extend this analysis:

1. **Day vs. Night temperatures**: Change `LST_Day_1km` to `LST_Night_1km`. Urban heat islands often persist overnight because concrete releases stored heat.

2. **Compare cities**: Try Phoenix (extreme heat island), Seattle (milder), or your hometown. Just change the TIGER filter.

3. **Historical comparison**: Compare 2024 to 2014—has development changed the heat pattern?

4. **Add more landmarks**: Identify specific neighborhoods to compare. North St. Louis historically has less tree canopy than Clayton—does that show up in the data?

5. **Environmental justice analysis**: Overlay census income data or tree canopy data to explore which communities bear the greatest heat burden.